In [22]:
import os
import pandas as pd
import re
from datetime import datetime

# Define a function for cleaning data
def clean_data(data, acct_cols):
    cleaned_data = data.copy()
    
    # Remove any leading or trailing whitespace in the account number column
    for col in acct_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

    # Check if any of the account number columns have invalid lengths
    for col in acct_cols:
        if cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)

    # Remove duplicates from the account number columns, if they exist
    if len(acct_cols) > 0:
        cleaned_data.drop_duplicates(subset=acct_cols, inplace=True)

        # Check if there are any duplicate account numbers after removing duplicates
        acct_counts = cleaned_data[acct_cols].value_counts()
        duplicates = acct_counts[acct_counts > 1].reset_index()[acct_cols]
        if len(duplicates) > 0:
            print("Warning: Duplicate account numbers found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=acct_cols, keep=False, inplace=True)
    
    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)
    
    return cleaned_data


# Common account number column names
acct_col_names = ["ACCT#", "ac_no", "Account_Number", "ACCOUNT_NUMBER", "ACCT_NO", "ACC_NO"]

# Ask the user to input the path to the folder containing the CSV or XLSX files they want to clean
search_path = input("Enter the path to search for CSV or XLSX files: ")
exts = (".csv", ".xlsx")
filenames = []
for root, dirs, files in os.walk(search_path):
    if os.path.basename(root) == "cleaned":
        continue
    for file in files:
        if file.endswith(exts):
            filenames.append(os.path.join(root, file))

# Create a "cleaned" folder if it doesn't exist
cleaned_folder = os.path.join(search_path, "cleaned")
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Clean each file individually
for filename in filenames:
    # Read the file into a Pandas DataFrame
    if filename.endswith(".csv"):
        data = pd.read_csv(filename)
    elif filename.endswith(".xlsx"):
        data = pd.read_excel(filename)
    else:
        print(f"Invalid file type. Skipping {filename}")
        continue
    
    # Check if any of the common account number column names are present
    acct_cols = []
    for col in data.columns:
        if col in acct_col_names:
            acct_cols.append(col)
    
    # Clean the data using the function we defined earlier, if an account number column is found
    if len(acct_cols) > 0:
        cleaned_data = clean_data(data, acct_cols)
    else:
        print(f"No account number column found in {filename}")
        cleaned_data = data
    
    # Normalize the file name and add date and time to the cleaned file name
    normalized_name = re.sub(r'\s+', '_', os.path.splitext(os.path.basename(filename))[0]).lower()
    cleaned_filename = f"{normalized_name}_cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    cleaned_filename = os.path.join(cleaned_folder, cleaned_filename)
    
    # Save the cleaned data to a new file
    cleaned_data.to_csv(cleaned_filename, index=False)
    print(f"{filename} cleaned and saved to {cleaned_filename}")


In [26]:
import os
import pandas as pd
import re
from datetime import datetime

# Define a function for cleaning data
def clean_data(data, acct_cols, cust_id_cols):
    cleaned_data = data.copy()
    
    # Remove any leading or trailing whitespace in the account number column
    for col in acct_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

        # Check if any of the account number columns have invalid lengths
        if cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)

    # Remove duplicates from the account number columns, if they exist
    if len(acct_cols) > 0:
        cleaned_data.drop_duplicates(subset=acct_cols, inplace=True)

        # Check if there are any duplicate account numbers after removing duplicates
        acct_counts = cleaned_data[acct_cols].value_counts()
        duplicates = acct_counts[acct_counts > 1].reset_index()[acct_cols]
        if len(duplicates) > 0:
            print("Warning: Duplicate account numbers found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=acct_cols, keep=False, inplace=True)

    # Clean the customer id column
    for col in cust_id_cols:
        if cleaned_data[col].str.contains("-").any():
            cleaned_data[col] = cleaned_data[col].str.split("-").str[-1].str.strip().str.replace(" ", "")
    
    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)
    
    return cleaned_data


# Common account number column names
acct_col_names = ["ACCT#", "ac_no", "Account_Number", "ACCOUNT_NUMBER", "ACCT_NO", "ACC_NO"]
cust_id_col_names = ["customer_id", "customerid", "customer_id_no", "CUSTOMER_ID", "customerid", "CustomerId", "CUST_ID"]

# Ask the user to input the path to the folder containing the CSV or XLSX files they want to clean
search_path = input("Enter the path to search for CSV or XLSX files: ")
exts = (".csv", ".xlsx")
filenames = []
for root, dirs, files in os.walk(search_path):
    if os.path.basename(root) == "cleaned":
        continue
    for file in files:
        if file.endswith(exts):
            filenames.append(os.path.join(root, file))

# Create a "cleaned" folder if it doesn't exist
cleaned_folder = os.path.join(search_path, "cleaned")
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Clean each file individually
for filename in filenames:
    # Read the file into a Pandas DataFrame
    if filename.endswith(".csv"):
        data = pd.read_csv(filename)
    elif filename.endswith(".xlsx"):
        data = pd.read_excel(filename)
    else:
        print(f"Invalid file type. Skipping {filename}")
        continue
    
    # Check if any of the common account number or customer id column names are present
    acct_cols = []
    cust_id_cols = []
    for col in data.columns:
        if col in acct_col_names:
            acct_cols.append(col)
        elif "customer_id" in col.lower():
            cust_id_cols.append(col)

    # Clean the data using the function we defined earlier, if an account number column is found
    if len(acct_cols) > 0:
        cleaned_data = clean_data(data, acct_cols, cust_id_cols)
    else:
        print(f"No account number column found in {filename}")
        cleaned_data = data

    # Clean the customer id column if it exists
    if len(cust_id_cols) > 0:
        for col in cust_id_cols:
            cleaned_data[col] = cleaned_data[col].str.split("-").str[-1].str.replace(" ", "").str.strip()

    # Normalize the file name and add date and time to the cleaned file name
    normalized_name = re.sub(r'\s+', '_', os.path.splitext(os.path.basename(filename))[0]).lower()
    cleaned_filename = f"{normalized_name}_cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    cleaned_filename = os.path.join(cleaned_folder, cleaned_filename)

    # Save the cleaned data to a new file
    cleaned_data.to_csv(cleaned_filename, index=False)
    print(f"{filename} cleaned and saved to {cleaned_filename}")


C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned_filename.csv cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\cleaned_filename_cleaned_20230311_023123.csv
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Adenta 20230209.xlsx cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\qr_code_project_-_adenta_20230209_cleaned_20230311_023141.csv
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Legon 20230209.xlsx cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\qr_code_project_-_legon_20230209_cleaned_20230311_023206.csv


In [43]:
def clean_data(data, cols):
    cleaned_data = data.copy()

    # Remove any leading or trailing whitespace in the columns
    for col in cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

        # Check if any of the columns have invalid lengths
        if col.startswith("ACC_NO") and cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)

    # Remove duplicates from the columns, if they exist
    if len(cols) > 0:
        cleaned_data.drop_duplicates(subset=cols, inplace=True)

        # Check if there are any duplicate values after removing duplicates
        value_counts = cleaned_data[cols].apply(lambda x: tuple(x), axis=1).value_counts()
        duplicates = value_counts[value_counts > 1].reset_index().rename(columns={"index": "values"})
        if len(duplicates) > 0:
            print("Warning: Duplicate values found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=cols, keep=False, inplace=True)

    # Clean the columns
    for col in cols:
        if col.startswith("CUST_ID") and cleaned_data[col].str.contains("-").any():
            cleaned_data[col] = cleaned_data[col].str.split("-", n=1).str[1].str.strip()
            cleaned_data[col] = cleaned_data[col].str.replace(r"[^0-9]+", "", regex=True)
            has_letters = cleaned_data[col].str.contains(r"[a-zA-Z]+")
            cleaned_data.loc[has_letters, col] = cleaned_data[col][has_letters]
        elif col.startswith("TEL"):
            cleaned_data[col] = cleaned_data[col].astype(str).str.replace("[^0-9]", "", regex=True)
            cleaned_data[col] = cleaned_data[col].apply(lambda x: "" if len(x) < 9 or x.count("0") > 3 else x)

    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)

    return cleaned_data

data = pd.read_excel("QR Code project - Adenta 20230209.xlsx")
cols = ["ACC_NO", "CUST_ID", "TEL"]
cleaned_data = clean_data(data, cols)
cleaned_data.to_csv("cleaned_filenamee.csv", index=False)


In [45]:
def clean_data(data, cols):
    cleaned_data = data.copy()

    # Remove any leading or trailing whitespace in the columns
    for col in cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

        # Check if any of the columns have invalid lengths
        if col.startswith("ACC_NO") and cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)

    # Remove duplicates from the columns, if they exist
    if len(cols) > 0:
        cleaned_data.drop_duplicates(subset=cols, inplace=True)

        # Check if there are any duplicate values after removing duplicates
        value_counts = cleaned_data[cols].apply(lambda x: tuple(x), axis=1).value_counts()
        duplicates = value_counts[value_counts > 1].reset_index().rename(columns={"index": "values"})
        if len(duplicates) > 0:
            print("Warning: Duplicate values found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=cols, keep=False, inplace=True)

    # Clean the columns
    for col in cols:
        if col.startswith("CUST_ID") and cleaned_data[col].str.contains("-").any():
            cleaned_data[col] = cleaned_data[col].str.split("-", n=1).str[1].str.strip()
            cleaned_data[col] = cleaned_data[col].str.replace(r"[^0-9]+", "", regex=True)
            has_letters = cleaned_data[col].str.contains(r"[a-zA-Z]+")
            cleaned_data.loc[has_letters, col] = cleaned_data[col][has_letters]
        elif col.startswith("TEL"):
            cleaned_data[col] = cleaned_data[col].astype(str).str.replace("[^0-9]", "", regex=True)
            cleaned_data[col] = cleaned_data[col].apply(lambda x: "" if len(x) < 9 or x.count("0") > 3 else x)

    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)

    return cleaned_data

# Ask the user to input the path to the folder containing the CSV or XLSX files they want to clean
search_path = input("Enter the path to search for CSV or XLSX files: ")
exts = (".csv", ".xlsx")
filenames = []
for root, dirs, files in os.walk(search_path):
    if os.path.basename(root) == "cleaned":
        continue
    for file in files:
        if file.endswith(exts):
            filenames.append(os.path.join(root, file))

# Create a "cleaned" folder if it doesn't exist
cleaned_folder = os.path.join(search_path, "cleaned")
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Clean each file individually
for filename in filenames:
    # Read the file into a Pandas DataFrame
    if filename.endswith(".csv"):
        data = pd.read_csv(filename)
    elif filename.endswith(".xlsx"):
        data = pd.read_excel(filename)
    else:
        print(f"Invalid file type. Skipping {filename}")
        continue
    
    # Check if any of the common account number or customer id column names are present
    acct_cols = []
    cust_id_cols = []
    phone_cols = []
    for col in data.columns:
        if col.upper() in acct_col_names:
            acct_cols.append(col)
        elif "CUSTOMER" in col.upper() and "ID" in col.upper():
            cust_id_cols.append(col)
        elif "PHONE" in col.upper() or "TELEPHONE" in col.upper():
            phone_cols.append(col)

    # Clean the data using the function we defined earlier, if an account number or customer id or phone number column is found
    if len(acct_cols) > 0 or len(cust_id_cols) > 0 or len(phone_cols) > 0:
        cleaned_data = clean_data(data, acct_cols, cust_id_cols, phone_cols)
    else:
        print(f"No account number, customer id or phone number column found in {filename}")
        cleaned_data = data

    # Normalize the file name and add date and time to the cleaned file name
    normalized_name = re.sub(r'\s+', '_', os.path.splitext(os.path.basename(filename))[0]).lower()
    cleaned_filename = f"{normalized_name}_cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    cleaned_filename = os.path.join(cleaned_folder, cleaned_filename)

    # Save the cleaned data to a new file
    cleaned_data.to_csv(cleaned_filename, index=False)
    print(f"{filename} cleaned and saved to {cleaned_filename}")


TypeError: clean_data() takes 2 positional arguments but 4 were given

In [49]:
def clean_data(data, acct_cols, cust_id_cols, phone_cols):
    cleaned_data = data.copy()

    # Define variables to track counts of changes made
    acct_invalid_lengths = 0
    acct_duplicates_removed = 0
    cust_id_values_changed = 0
    phone_no_values_changed = 0

    # Remove any leading or trailing whitespace in the account number column
    for col in acct_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

        # Check if any of the account number columns have invalid lengths
        if cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)
            acct_invalid_lengths += len(invalid_acct_nums)

    # Remove duplicates from the account number columns, if they exist
    if len(acct_cols) > 0:
        cleaned_data.drop_duplicates(subset=acct_cols, inplace=True)

        # Check if there are any duplicate account numbers after removing duplicates
        acct_counts = cleaned_data[acct_cols].value_counts()
        duplicates = acct_counts[acct_counts > 1].reset_index()[acct_cols]
        if len(duplicates) > 0:
            print("Warning: Duplicate account numbers found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=acct_cols, keep=False, inplace=True)
            acct_duplicates_removed += len(duplicates)

    # Clean the customer id column
    for col in cust_id_cols:
        if cleaned_data[col].str.contains("-").any():
            cleaned_data[col] = cleaned_data[col].str.split("-", n=1).str[1].str.strip()
            cleaned_data[col] = cleaned_data[col].str.replace(r"[^0-9]+", "", regex=True)
            has_letters = cleaned_data[col].str.contains(r"[a-zA-Z]+")
            cleaned_data.loc[has_letters, col] = cleaned_data[col][has_letters]
            cust_id_values_changed += sum(has_letters)

    # Clean the phone number column
    for col in phone_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.replace("[^0-9]", "", regex=True)
        cleaned_data[col] = cleaned_data[col].apply(lambda x: "" if len(x) < 9 or x.count("0") > 3 or x.count("00") > 0 else x)
        phone_no_values_changed += len(cleaned_data[col]) - cleaned_data[col].astype(bool).sum(axis=0)

    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)

    # Print the report
    print(f"\nReport for {os.path.basename(filename)}:")
    print(f"Number of invalid account number lengths removed: {acct_invalid_lengths}")
    print(f"Number of duplicate account numbers removed: {acct_duplicates_removed}")
    print(f"Number of customer ID values changed: {cust_id_values_changed}")
    print(f"Number of phone number values changed: {phone_no_values_changed}")

    return cleaned_data

acct_col_names = ["ACC_NO"]
cust_id_col_names = ["CUST_ID"]
phone_no_col_names = ["TEL"]

# Ask the user to input the path to the folder containing the CSV or XLSX files they want to clean
search_path = input("Enter the path to search for CSV or XLSX files: ")
exts = (".csv", ".xlsx")
filenames = []
for root, dirs, files in os.walk(search_path):
    if os.path.basename(root) == "cleaned":
        continue
    for file in files:
        if file.endswith(exts):
            filenames.append(os.path.join(root, file))

# Create a "cleaned" folder if it doesn't exist
cleaned_folder = os.path.join(search_path, "cleaned")
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Clean each file individually
for filename in filenames:
    # Read the file into a Pandas DataFrame
    if filename.endswith(".csv"):
        data = pd.read_csv(filename)
    elif filename.endswith(".xlsx"):
        data = pd.read_excel(filename)
    else:
        print(f"Invalid file type. Skipping {filename}")
        continue
    
    # Check if any of the common account number or customer id column names are present
    acct_cols = []
    cust_id_cols = []
    phone_cols = []
    for col in data.columns:
        if col in acct_col_names:
            acct_cols.append(col)
        elif col in cust_id_col_names:
            cust_id_cols.append(col)
        elif col in phone_no_col_names:
            phone_cols.append(col)

    # Clean the data using the function we defined earlier, if an account number or customer id or phone number column is found
    if len(acct_cols) > 0 or len(cust_id_cols) > 0 or len(phone_cols) > 0:
        cleaned_data = clean_data(data, acct_cols, cust_id_cols, phone_cols)
    else:
        print(f"No account number, customer id or phone number column found in {filename}")
        cleaned_data = data

    # Normalize the file name and add date and time to the cleaned file name
    normalized_name = re.sub(r'\s+', '_', os.path.splitext(os.path.basename(filename))[0]).lower()
    cleaned_filename = f"{normalized_name}_cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    cleaned_filename = os.path.join(cleaned_folder, cleaned_filename)

    # Save the cleaned data to a new file
    cleaned_data.to_csv(cleaned_filename, index=False)
    print(f"{filename} cleaned and saved to {cleaned_filename}")



Report for QR Code project - Adenta 20230209.xlsx:
Number of invalid account number lengths removed: 0
Number of duplicate account numbers removed: 0
Number of customer ID values changed: 0
Number of phone number values changed: 33698
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Adenta 20230209.xlsx cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\qr_code_project_-_adenta_20230209_cleaned_20230311_101703.csv

Report for QR Code project - Legon 20230209.xlsx:
Number of invalid account number lengths removed: 0
Number of duplicate account numbers removed: 0
Number of customer ID values changed: 0
Number of phone number values changed: 60424
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Legon 20230209.xlsx cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\qr_code_project_

In [55]:
def clean_data(data, acct_cols, cust_id_cols, phone_cols):
    cleaned_data = data.copy()
    
    # Initialize counters
    num_invalid_acct_nums = 0
    num_duplicates_removed = 0
    num_phone_numbers_cleaned = 0

    # Remove any leading or trailing whitespace in the account number column
    for col in acct_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.strip()

        # Check if any of the account number columns have invalid lengths
        if cleaned_data[col].str.replace(" ", "").str.len().ne(9).any():
            print(f"Warning: Account numbers in column '{col}' have invalid lengths and will be removed:")
            invalid_acct_nums = cleaned_data[col][cleaned_data[col].str.replace(" ", "").str.len().ne(9)]
            print(invalid_acct_nums.to_string(index=False))
            cleaned_data.drop(index=invalid_acct_nums.index, inplace=True)
            num_invalid_acct_nums += len(invalid_acct_nums)

    # Remove duplicates from the account number columns, if they exist
    if len(acct_cols) > 0:
        cleaned_data.drop_duplicates(subset=acct_cols, inplace=True)

        # Check if there are any duplicate account numbers after removing duplicates
        acct_counts = cleaned_data[acct_cols].value_counts()
        duplicates = acct_counts[acct_counts > 1].reset_index()[acct_cols]
        if len(duplicates) > 0:
            print("Warning: Duplicate account numbers found in the following rows and will be removed:")
            print(duplicates.to_string(index=False))
            cleaned_data.drop_duplicates(subset=acct_cols, keep=False, inplace=True)
            num_duplicates_removed += len(duplicates)

    # Clean the customer id column
    for col in cust_id_cols:
        if cleaned_data[col].str.contains("-").any():
            cleaned_data[col] = cleaned_data[col].str.split("-", n=1).str[1].str.strip()
            cleaned_data[col] = cleaned_data[col].str.replace(r"[^0-9]+", "", regex=True)
            has_letters = cleaned_data[col].str.contains(r"[a-zA-Z]+")
            cleaned_data.loc[has_letters, col] = cleaned_data[col][has_letters]

    # Clean the phone number column
    for col in phone_cols:
        cleaned_data[col] = cleaned_data[col].astype(str).str.replace("[^0-9]", "", regex=True)
        num_phone_numbers_cleaned += (cleaned_data[col].str.len() >= 9).sum()
        cleaned_data[col] = cleaned_data[col].apply(lambda x: "" if len(x) < 9 or x.count("0") > 3 else x)
    
    # Reset the index after cleaning
    cleaned_data.reset_index(drop=True, inplace=True)

    # Print report
    print(f"Processed {len(data)} rows in {len(acct_cols) + len(cust_id_cols) + len(phone_cols)} columns")
    print(f"Number of invalid account numbers removed: {num_invalid_acct_nums}")
    print(f"Number of duplicate rows removed: {num_duplicates_removed}")
    print(f"Number of phone numbers cleaned: {num_phone_numbers_cleaned}")
    
    return cleaned_data


acct_col_names = ["ACC_NO"]
cust_id_col_names = ["CUST_ID"]
phone_no_col_names = ["TEL"]

# Ask the user to input the path to the folder containing the CSV or XLSX files they want to clean
search_path = input("Enter the path to search for CSV or XLSX files: ")
exts = (".csv", ".xlsx")
filenames = []
for root, dirs, files in os.walk(search_path):
    if os.path.basename(root) == "cleaned":
        continue
    for file in files:
        if file.endswith(exts):
            filenames.append(os.path.join(root, file))

# Create a "cleaned" folder if it doesn't exist
cleaned_folder = os.path.join(search_path, "cleaned")
if not os.path.exists(cleaned_folder):
    os.makedirs(cleaned_folder)

# Clean each file individually
for filename in filenames:
    # Read the file into a Pandas DataFrame
    if filename.endswith(".csv"):
        data = pd.read_csv(filename)
    elif filename.endswith(".xlsx"):
        data = pd.read_excel(filename)
    else:
        print(f"Invalid file type. Skipping {filename}")
        continue
    
    # Check if any of the common account number or customer id column names are present
    acct_cols = []
    cust_id_cols = []
    phone_cols = []
    for col in data.columns:
        if col in acct_col_names:
            acct_cols.append(col)
        elif col in cust_id_col_names:
            cust_id_cols.append(col)
        elif col in phone_no_col_names:
            phone_cols.append(col)
    
    if len(acct_cols) > 0 or len(cust_id_cols) > 0 or len(phone_cols) > 0:
        cleaned_data = clean_data(data, acct_cols, cust_id_cols, phone_cols)
    else:
        print(f"No account number, customer id or phone number column found in {filename}")
        cleaned_data = data

    normalized_name = re.sub(r'\s+', '_', os.path.splitext(os.path.basename(filename))[0]).lower()
    cleaned_filename = f"{normalized_name}_cleaned_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    cleaned_filename = os.path.join(cleaned_folder, cleaned_filename)

    cleaned_data.to_csv(cleaned_filename, index=False)

    print(f"{filename} cleaned and saved to {cleaned_filename}")

       # Print report on changes made to the file
    print("\nCleaned data report:")
    print("====================")
    print(f"File name: {filename}")
    print(f"Number of rows before cleaning: {len(data)}")
    print(f"Number of rows after cleaning: {len(cleaned_data)}")

    # Account number
    if len(acct_cols) > 0:
        removed_accounts = len(data) - len(cleaned_data)
        if removed_accounts > 0:
            print(f"Removed {removed_accounts} rows with invalid account numbers.")
        acct_counts_before = data[acct_cols].nunique().values
        acct_counts_after = cleaned_data[acct_cols].nunique().values
        diff = acct_counts_before - acct_counts_after
        if (diff > 0).any():
            print(f"Removed {diff.sum()} duplicate account numbers.")
            for i, col in enumerate(acct_cols):
                if diff[i] > 0:
                    print(f"  {diff[i]} duplicates removed from column '{col}'.")
    else:
        print("No account number column found.")

    # if len(cust_id_cols) > 0:
    #     common_cols = list(set(cust_id_cols).intersection(set(data.columns)))
    #     data_common, cleaned_common = data[common_cols], cleaned_data[common_cols]
    #     data_common, cleaned_common = data_common.align(cleaned_common)
    #     changed_cust_ids = (cleaned_common != data_common).any(axis=1).sum()
    #     if changed_cust_ids > 0:
    #         print(f"Changed {changed_cust_ids} customer IDs.")

    
    # # Phone number
    # if len(phone_cols) > 0:
    #     removed_phone_nums = (data[phone_cols].astype(str) != cleaned_data[phone_cols].astype(str)).any(axis=1).sum()
    #     if removed_phone_nums > 0:
    #         print(f"Removed {removed_phone_nums} rows with invalid phone numbers.")
    # else:
    #     print("No phone number column found.")


Processed 96102 rows in 3 columns
Number of invalid account numbers removed: 0
Number of duplicate rows removed: 0
Number of phone numbers cleaned: 71047
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Adenta 20230209.xlsx cleaned and saved to C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\cleaned\qr_code_project_-_adenta_20230209_cleaned_20230311_174115.csv

Cleaned data report:
File name: C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Adenta 20230209.xlsx
Number of rows before cleaning: 96102
Number of rows after cleaning: 96100
Removed 2 rows with invalid account numbers.
Processed 121466 rows in 3 columns
Number of invalid account numbers removed: 0
Number of duplicate rows removed: 0
Number of phone numbers cleaned: 81431
C:\Users\Moshood\OneDrive\dev\python_data_profiling_scripts\base_data_cleaning\QR Code project - Legon 20230209.xlsx cleaned and sav